In [1]:
import json
import spacy
import pandas as pd
from spacy import displacy

In [2]:
file = open("example.json", "r", encoding="utf8")
strJson = file.readline()

In [3]:
json_data = json.loads(strJson)

In [4]:
print(json_data)

{'content': 'Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• 

In [5]:
file = open("example.json", "r" , encoding="utf8")
json1 = file.readlines()
datas = [json.loads(x) for x in json1]

In [6]:
#spacy.cli.download("en_core_web_sm")

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
test_data = datas.pop()
test_data = test_data["content"]
print(test_data)

Harini Komaravelli
Test Analyst at Oracle, Hyderabad

Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini-
Komaravelli/2659eee82e435d1b

➢ 6 Yrs. of IT Experience in Manual and Automation testing.

WORK EXPERIENCE

QA Analyst

Oracle

Test Analyst at Oracle, Hyderabad

Infosys Ltd -  Hyderabad, Telangana -

November 2011 to February 2016

Hyderabad from Nov 2011 to Feb17 2016
➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017
➢ Currently working as a Test Analyst at Oracle, Hyderabad

QA Analyst with 6 years of IT experience

Oracle

EDUCATION

MCA

Osmania University

B.Sc. in Computer Science

Osmania University

SKILLS

Functional Testing, Blue Prism, Qtp

ADDITIONAL INFORMATION

Area of Expertise:

➢ Familiar with Agile Methodologies.
➢ Having knowledge in Energy (Petroleum) & Health Care domains.
➢ Involved in preparation of Test Scenarios.
➢ Preparing Test Data for the test cases.

https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?is

In [9]:
nlp.remove_pipe("ner")

('ner', <spacy.pipeline.ner.EntityRecognizer at 0x13737b74890>)

In [10]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x13737c70b80>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x13737b90220>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x13737af3890>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x13737e04cc0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x13737e97bc0>)]

Name: spacy
Version: 3.2.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: c:\users\burak\anaconda3\lib\site-packages
Requires: blis, preshed, thinc, wasabi, typer, pathy, jinja2, spacy-loggers, packaging, srsly, spacy-legacy, setuptools, requests, murmurhash, langcodes, numpy, tqdm, cymem, pydantic, catalogue
Required-by: en-core-web-sm, en-core-web-md, en-core-web-lg


In [11]:
TRAIN_DATA = []
for data in datas:
    myList = []
    myList.append(data["content"])
    entities = dict()
    entities["entities"] = []
    for entity in data["annotation"]:
        if not entity["label"]:
            continue
        entities["entities"].append((entity["points"][0]["start"], entity["points"][0]["end"], entity["label"][0]))
    myList.append(entities)
    TRAIN_DATA.append(tuple(myList))
TRAIN_DATA

[('Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programmin

In [12]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("en")
    ner = nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

In [13]:
import random
import datetime as dt
from spacy.util import minibatch, compounding
from spacy.training.example import Example

nlp = create_blank_nlp(TRAIN_DATA)
examples = []
for text, annots in TRAIN_DATA:
    examples.append(Example.from_dict(nlp.make_doc(text), annots))
nlp.initialize(lambda: examples)
for i in range(20):
    losses = {}
    batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        nlp.update(
            batch,
            drop=0.1,
            losses=losses,
        )
    print(f"Losses at iteration {i} - {dt.datetime.now()} {losses}")

C:\Users\Burak\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with entities "[(1155, 1198, 'Email Address'), (743, 1140, 'Skill...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Akhil Yadav Polemaina
Hyderabad, Telangana - Email..." with entities "[(3749, 3756, 'Skills'), (3709, 3717, 'Skills'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\Burak\anaconda3\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities coul

ValueError: [E103] Trying to set conflicting doc.ents: '(1209, 1214, 'Companies worked at')' and '(1136, 1247, 'Skills')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.

In [ ]:
nlp.pipeline

In [ ]:
test_data

In [ ]:
doc = nlp(test_data)
displacy.render(doc, style="ent")

In [1]:
print("Anurag Asthana\nPune, Maharashtra - Email me on Indeed: indeed.com/r/Anurag-Asthana/ea7451b2bdb6115a\n\n• Looking forward for joining a company where my creative thinking and excellent skills in\napplication development using modern development tools will be utilized.\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nApril 2017 to Present\n\nLocation: Pune, India\nClient: Insurance Client from London\nDuration: April 2017 -Till Date\nTechnology/Tools: Microsoft SharePoint 2013, Azure Cognitive Services, Azure Service Bus, Azure\nBlob Storage, .net framework 4.7\n\nResponsibilities:\n\n• Independently worked on designing solution architecture of the project.\n• Lead team from technical front in all the components built\n• Followed Agile development approach to efficiently manage continuous development process\nand incremental requirement changes\n• Design SharePoint components using SharePoint CSOM and .net framework 4.7\n• Design Azure components using azure .net sdks and .net framework 4.7\n• Implement Micro Services architecture using Azure Service Bus\n\nProject Title: SharePoint 2013 Application Development and Enhancement\n\nInfosys LTD -  London -\n\nFebruary 2017 to March 2018\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nAugust 2015 to January 2017\n\nSharePoint Designer\n\nMicrosoft SharePoint -\n\n2017 to 2017\n\n2013, Nintex.\n\nResponsibilities:\n• Investigated and identified requirements via process flows, use cases communicated with\nSubject Matter Experts to define system requirement.\n\nhttps://www.indeed.com/r/Anurag-Asthana/ea7451b2bdb6115a?isid=rex-download&ikw=download-top&co=IN\n\n\n• Suggested improvements to be included into new system for Skills Alignment Portal\n• Followed Agile development approach to efficiently manage continuous development process\nand incremental requirement changes\n• Independently designed, developed and delivered multiple forms and workflows over\nSharePoint 2013 start from scratch using Nintex\n• Designed and Implemented useful components using SharePoint out-of-the-box capabilities and\nSharePoint REST web service\n• Utilized SharePoint APPs technology to create custom enhancements on current sites for\ncustomer satisfaction and efficiency\n• Developed underwriting support applications using asp.net MVC / Web API.\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nDecember 2013 to July 2015\n\nEnterprise IT Support\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nSeptember 2013 to November 2013\n\nLocation: Pune, India\nClient: Oil and Gas Client from USA\nDuration: September 2013 - November 2013\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nJune 2013 to August 2013\n\nLocation: Pune, India\nClient: Oil and Gas Client from USA\nDuration: June 2013 - August 2013\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nFebruary 2013 to May 2013\n\nInfosys LTD -  MAHARASHTRA, IN -\n\nNovember 2012 to January 2013\n\nLocation: Pune, India\nClient: Infosys Ltd.\nDuration: November 2012 - January 2013\n\nSharePoint Designer, Avepoint Docave\n\nMicrosoft SharePoint -\n\n2012 to 2012\n\n6.\n\nResponsibilities:\n\n\n\n• Responsible for creating SharePoint hosted apps.\n• Responsible for developing web services [WEB API's].\n• Responsible for creating ECMA scripts.\n• Responsible for creating console applications using SharePoint Client Object Model.\n• Responsible for migrating SharePoint MOSS 2007 sites to SharePoint 2013 using Avepoint\nDocave.\n• Responsible for developing remote site provisioning mechanism using CSOM\n• Responsible for creating provider hosted apps using asp.net mvc.\n\nProject Title: Operating System\n\nMicrosoft Fast Search, SharePoint Designer\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\nResponsibilities:\n\n• Responsible for maintaining database of the project.\n• Responsible for designing page layouts.\n• Responsible for creating ECMA scripts.\n• Responsible for creating console applications using SharePoint Client Object Model.\n• Responsible for creating SSIS solution to import data from multiple sources using SharePoint\nweb services.\n• Responsible for creating custom SharePoint alerts using SSIS solution.\n• Responsible for creating SSRS solution to create Dashboards.\n• Responsible for creating Stored Procedures and Functions used for various operations.\n• Responsible for customizing list forms.\n• Responsible for creating Search pages using Microsoft FAST search 2010.\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\nResponsibilities:\n\n• Responsible for fixing bugs in SharePoint applications\n• Responsible for automating bug fixing process.\n• Responsible for interacting with clients for understanding business requirements.\n• Responsible for enhancing SharePoint application as per business requirements.\n• Identifying limitations and suggesting appropriate solutions.\n\nMicrosoft InfoPath Designer\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\n2010, Microsoft SharePoint Designer 2010, Microsoft Office 2010.\n\nResponsibilities:\n\n\n\n• Responsible for creating multiple custom lists and document libraries.\n• Responsible for creating and customizing InfoPath forms and associating it with document\nlibraries.\n• Responsible for creating SharePoint Designer Workflows.\n• Responsible for customizing list forms and views.\n• Responsible for creating Excel reports in SharePoint.\n\nProject Title: 5 Day Close Monitoring Tool\n\nTechnology/Tools: Microsoft SharePoint 2010, Microsoft SQL Server 2008 R2, Microsoft Visual\nStudio 2010, Microsoft InfoPath Designer 2010, Microsoft SharePoint Designer 2010, Microsoft\nOffice 2010\nResponsibilities:\n\n• Responsible for creating custom lists, document libraries, SharePoint groups.\n• Responsible for creating custom permission levels in SharePoint and defining permission\npolicies on lists, libraries and web parts.\n• Responsible for maintaining database of the project.\n• Responsible to create SSIS packages.\n• Responsible to create power pivot reports.\n• Responsible for creating dashboards on SharePoint site using Power Pivot library.\n• Responsible for creating tool for generating power pivot reports on multiple servers.\n\nMicrosoft Designer\n\nMicrosoft SharePoint -\n\n2010 to 2010\n\n2010.\nResponsibilities:\n\n• Responsible for creating multiple custom lists and document libraries.\n• Responsible for creating and customizing InfoPath forms and associating it with document\nlibraries.\n• Responsible for creating SharePoint Designer Workflows.\n• Responsible for customizing list forms and views.\n• Responsible for creating custom permission levels and applying it to various lists and libraries.\n\nEDUCATION\n\nBachelor of Technology in Information Technology\n\nGraphic Era University -  Dehra Dun, Uttarakhand\n\nMay 2012\n\nSenior Secondary -  Kashipur, Uttarakhand\n\n2008\n\n\n\nSKILLS\n\nMICROSOFT SHAREPOINT (7 years), SHAREPOINT (7 years), .NET (6 years), MICROSOFT VISUAL\nSTUDIO (6 years), VISUAL STUDIO (6 years)\n\nADDITIONAL INFORMATION\n\nSKILLS\n• Vast knowledge of developing and implementing applications based on client's needs.\n• Professional Experience in creating SharePoint solutions using C#, JavaScript etc.\n• Professional Experience in creating asp.net web applications using MVC, Entity Framework.\n• Professional experience in creating application using Azure micro services architecture\n\nTECHNICAL EXPERTISE\n\nLanguages and Software\nC, C#.net, Asp.net MVC 5, Entity Framework 6, Microsoft Office, Microsoft SharePoint 2010,\nMicrosoft SharePoint 2013, Microsoft, SharePoint Designer, Microsoft Visual Studio 2008,\nMicrosoft Visual Studio 2010, Microsoft Visual Studio 2012, InfoPath, HTML, CSS, JavaScript,\njQuery, SharePoint Apps using MVC and KnockoutJS, Nintex forms and Workflows, Azure\n\nMSBI SQL Server 2008, SQL Server 2012, SSIS, SSRS\nOperating System Windows Family\n\nTOTAL EXPERIENCE\n\nNo. Company Role Duration\n1. Infosys Ltd Technology Analyst 5 Years 11 Months")

Anurag Asthana
Pune, Maharashtra - Email me on Indeed: indeed.com/r/Anurag-Asthana/ea7451b2bdb6115a

• Looking forward for joining a company where my creative thinking and excellent skills in
application development using modern development tools will be utilized.

Willing to relocate: Anywhere

WORK EXPERIENCE

Infosys LTD -  MAHARASHTRA, IN -

April 2017 to Present

Location: Pune, India
Client: Insurance Client from London
Duration: April 2017 -Till Date
Technology/Tools: Microsoft SharePoint 2013, Azure Cognitive Services, Azure Service Bus, Azure
Blob Storage, .net framework 4.7

Responsibilities:

• Independently worked on designing solution architecture of the project.
• Lead team from technical front in all the components built
• Followed Agile development approach to efficiently manage continuous development process
and incremental requirement changes
• Design SharePoint components using SharePoint CSOM and .net framework 4.7
• Design Azure components using azure .net sdks and

In [2]:
print("Hemil Bhavsar\nJr. ASP.NET Developer in True Vision Technology\n\nAhmedabad, Gujarat - Email me on Indeed: indeed.com/r/Hemil-Bhavsar/ce3a928d837ce9e1\n\nTo seek a job in the IT industry where I can utilize my skills and technical knowledge for the\norganization's and individual growth\n\nArea of interest:\n• Software Testing.\n• Testing application manually.\n• Great interest to learn new Technologies and Languages.\n• Enterprise Application Development\n• Web Application Development\n\nWORK EXPERIENCE\n\nJr. ASP.NET Developer in True Vision Technology\n\n-\n\nSeptember 2015 to May 2016\n\nSoftware Testing Training at Unicode Technology From March 2018 to May 2018\n\nJr. ASP.NET Developer (Intern) in Softech Infosys\n\n-\n\nMarch 2015 to August 2015\n\nEDUCATION\n\nMSC\n\nKSV University -  Gandhinagar, Gujarat\n\nApril 2015\n\nB.C.A\n\nGujarat University\n\nApril 2013\n\nSKILLS\n\nASP (1 year), ASP.NET (1 year), MS ASP (1 year), SOFTWARE TESTING (Less than 1 year),\nTESTING (Less than 1 year)\n\nhttps://www.indeed.com/r/Hemil-Bhavsar/ce3a928d837ce9e1?isid=rex-download&ikw=download-top&co=IN\n\n\nADDITIONAL INFORMATION\n\nTechnical skillset:\n• Manual Testing Techniques\n• Web Application testing\n• STLC (Software Testing Life Cycle) process\n• Verification and Validation process\n• Programming Language: Software Testing, ASP.Net with C#\n\n• Frameworks: Microsoft Visual Studio 2010, Frameworks 3.5, Frameworks (C#) 4.0, 4.5\n• Database tools: MySQL, SQL Server 2008, SQL Server 2010, MS-Access 2008\n• Report generation Tool: Crystal Report")

Hemil Bhavsar
Jr. ASP.NET Developer in True Vision Technology

Ahmedabad, Gujarat - Email me on Indeed: indeed.com/r/Hemil-Bhavsar/ce3a928d837ce9e1

To seek a job in the IT industry where I can utilize my skills and technical knowledge for the
organization's and individual growth

Area of interest:
• Software Testing.
• Testing application manually.
• Great interest to learn new Technologies and Languages.
• Enterprise Application Development
• Web Application Development

WORK EXPERIENCE

Jr. ASP.NET Developer in True Vision Technology

-

September 2015 to May 2016

Software Testing Training at Unicode Technology From March 2018 to May 2018

Jr. ASP.NET Developer (Intern) in Softech Infosys

-

March 2015 to August 2015

EDUCATION

MSC

KSV University -  Gandhinagar, Gujarat

April 2015

B.C.A

Gujarat University

April 2013

SKILLS

ASP (1 year), ASP.NET (1 year), MS ASP (1 year), SOFTWARE TESTING (Less than 1 year),
TESTING (Less than 1 year)

https://www.indeed.com/r/Hemil-Bhavsar